<a href="https://colab.research.google.com/github/abyanrifqy/ProjectSingkat/blob/master/flask-colab/Povert_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install flask-ngrok

In [ ]:
import os
import io
import numpy as np
import json
import pandas as pd

from flask_ngrok import run_with_ngrok
from flask import Flask, request, redirect, url_for, jsonify

from keras import backend as K
import keras
from tensorflow import keras
import tensorflow as tf


app = Flask(__name__)
run_with_ngrok(app)

def load_model():
    global model
    global backends
    model_path = '/content/drive/Shareddrives/Capstone_Project/ML Poverty/Poverty_model_Cloud'
    model = keras.models.load_model(model_path)
    backends = K.get_session()

load_model()

names = ['extreme-poor','moderate-poor', 'non-vulnarable', 'vulnarable']


@app.route('/', methods=['GET', 'POST'])
def input_data():
        if request.method == 'POST':

            rooms = request.form.get('rooms') # total rooms
            v14a = request.form.get('v14a') # have toilet or not
            refrig = request.form.get('refrig') # have refiregerator or not
            television = request.form.get('television') # have television or not
            computer = request.form.get('computer') # have computer or not
            hogar_nin = request.form.get('hogar_nin') # Number of children age 0 to 19
            escolari = request.form.get('escolari') # Years of schooling
            dis = request.form.get('dis') # disable person
            estadocivil3 = request.form.get('estadocivil3') # married or not
            estadocivil4 = request.form.get('estadocivil4') # divorce or not
            hogar_mayor = request.form.get('hogar_mayor') # number of individuals age 65+ 
            hogar_total = request.form.get('hogar_total') # total individuals
            tipovivi3 = request.form.get('tipovivi3') # rent house or not
            v2a1 = request.form.get('v2a1') # monthly rent payment
            dependency = request.form.get('dependency') # dependency rate
            SQBdependency = float(dependency)**2 # dependency**2
            r4t1 = request.form.get('r4t1') # persons younger than 12 years

            # obtain output from room classification. Output must be converted into float64/float32
            bedroom = request.form.get('bedroom') 
            kitchen = request.form.get('kitchen') 
            livingroom = request.form.get('livingroom') 

            data = {
                  'rooms': rooms,
                  'v14a': v14a,
                  'refrig' : refrig,
                  'television' : television,
                  'computer':computer,
                  'hogar_nin' : hogar_nin,
                  'escolari' : escolari,
                  'dis':dis,
                  'estadocivil3' : estadocivil3,
                  'estadocivil4' : estadocivil4,
                  'hogar_mayor' : hogar_mayor,
                  'hogar_total' : hogar_total,
                  'tipovivi3': tipovivi3,
                  'v2a1' :v2a1,
                  'dependency' : dependency,
                  'SQBdependency': SQBdependency,
                  'r4t1' : r4t1,
                  'bedroom':bedroom,
                  'kitchen':kitchen,
                  'livingroom':livingroom
            }

            df = pd.DataFrame([data], columns=data.keys())

            global backends
            with backends.as_default():
              model_path = '/content/drive/Shareddrives/Capstone_Project/ML Poverty/Poverty_model_Cloud'
              model = keras.models.load_model(model_path)
              backends = K.get_session()
              # Data
              df = np.array(df)
              df = K.constant(df)
              print(df)
              prediction = model.predict(df)
              print(prediction)
              prediction_prob = prediction.max()
              print(prediction_prob)
              place = np.argmax(prediction)
              print(place)
              result = names[place]
            
              response_json = {
                # "data" : data, 
                "prediction": str(result),
                "probabilty": str(prediction_prob)
                }

            return jsonify(response_json)
      
        return '''
        <!doctype html>
        <title>Upload new form</title>
        <h1>Fill form </h1>
        <form method=post enctype=multipart/form-data>
          <p> Rooms <input type=number name= "rooms">
          <p> Toilet <input type=number name= "v14a">
          <p> Refrigerator <input type=number name= "refrig">
          <p> TV <input type=number name= "television">
          <p> Computer <input type=number name= "computer">
          <p> Number of children age 0 to 19 <input type=number name= 'hogar_nin'>
          <p> Years of schooling <input type=number name= "escolari">
          <p> Disable person <input type=number name= "dis">
          <p> Married <input type=number name= 'estadocivil3'>
          <p> Divorce <input type=number name= 'estadocivil4'>
          <p> Number of individuals age 65+ <input type=number name= 'hogar_mayor'>
          <p> Number of total individuals <input type=number name= 'hogar_total'>
          <p> Rent House/Not <input type=number name= 'tipovivi3'>
          <p> Monthly rent payment <input type=number name= 'v2a1'>
          <p> Dependency <input type=number name= 'dependency'>
          <p> Persons younger than 12 years <input type=number name= 'r4t1'>
          <p> bedroom <input type=number name= 'bedroom'>
          <p> kitchen <input type=number name= 'kitchen'>
          <p> livingroom <input type=number name= 'livingroom'>
          <p> <input type=submit value=input_data>
        </form>
          '''

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://231f2453be3d.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Jun/2021 08:58:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 08:58:08] "GET /favicon.ico HTTP/1.1" 404 -
[2021-06-09 08:58:44,169] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint]

tf.Tensor(
[[ 3.  2.  1.  2.  2.  1. 12.  0.  1.  0.  0.  4.  0.  0.  2.  4.  0.  1.
   0.  0.]], shape=(1, 20), dtype=float32)
[[0.11943253 0.02503859 0.84816027 0.00736861]]
0.84816027
2


[2021-06-09 08:59:06,475] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-7-3d6b4c12d6fe>", line 99, in input_data
    print(results)
NameError: name 'results' is n

tf.Tensor(
[[ 3.  2.  1.  2.  2.  1. 12.  0.  1.  0.  0.  4.  0.  0.  2.  4.  0.  1.
   0.  0.]], shape=(1, 20), dtype=float32)
[[0.11943253 0.02503859 0.84816027 0.00736861]]
0.84816027
2


0 : extreme-poor

1 : moderate-poor

2 : non-vulnarable

3 : vulnarable

Still lacking of object detection model (changed to fill it in the form). Not yet do filtering based on categories because doesn't have enough time. 